In [1]:
import numpy as np
import random
import math

from matplotlib import pyplot as plt

In [34]:
# Paramètres
n = m = 18 # nombre de lignes = nombre de colonnes
initial_size = 2 # taille de la fourmilière initiale
max_food_quantity = 20 #quantité maximale de nourriture par source de nourriture
neigh = 1 #voisinage / champ de vision de chaque fourmi
pers_pher = 100 #persistance des phéromones dans l'environnement
life_time = 200 #durée de vie initiale des fourmis
reward = 10 #bonus de durée de vie pour avoir trouvé de la nourriture
nb_ants = 8 #nombre de fourmis initiales
food_count = 0 #quantité initiale de nourriture apportée au nid
creation_rate = 6 # valeur de food_count pour laquelle une nouvelle fourmi est créée
exploration_rate = 10 #facteur d'exploration
max_turn = 200 #nombre de tours effectués
nb_trap = 10 #nombre de pièges
memory = 8 #mémoire des fourmis

In [35]:
def generate_simple_map(n, m, nb_trap): # n : lignes m : colonnes
    """Préconditions : m > 0 et n > 0, initial_size > 0
    Renvoie un monde de taille n*m avec une fourmilière au centre et un point de nourriture en périphérie"""
    map = np.array([[" " for i in range(0, n)] for j in range(0, n)]) # on crée la matrice vide (avec des " " partout)
    # on crée la fourmilière au centre
    for i in range(0, initial_size):
        for k in range(0, initial_size):
            map = np.delete(map, m*(n//2 - initial_size//2 + i) + m//2 - initial_size//2 + k) #on copie le monde en enlevant une à une les cases du carré de taille initial_size situé au centre
            map = np.insert(map, m*(n//2 - initial_size//2 + i) + m//2 - initial_size//2 + k, "h") #on remplace les cases initialement vide (avec un " ") par une case "fourmilière" (avec un "h" pour "home")
            map = np.array([[map[m*j + i] for i in range(0, m)] for j in range(0, n) ])
    
    # on crée un point de nourriture
    food_line = random.randint(0, 2*n//5 - 1) # on définit ainsi la périphérie comme étant les cases du bord de la matrice avec un largeur de n/5
    food_column = random.randint(0, 2*m//5 - 1)
    if food_line > n//5 - 1: # on est sur la périphérie du bas
        food_line = n - 2 - n//5 + food_line
    if food_column > m//5 - 1: # on est sur la périphérie de droite
        food_column = m - 2 - m//5 + food_column
    where = random.sample([food_line, food_column], 1) # on définit une périphérie où se mettre (horizontale ou verticale)
    if where == food_line: # on est sur une périphérie en haut ou en bas
        food_column = random.randint(0, m - 1) # on définit une case au hasard sur cette périphérie
    else : #on est sur une périphérie latérale
        food_line = random.randint(0, n - 1) # on définit une case au hasard sur cette périphérie
    map = np.delete(map, m*food_line + food_column)
    map = np.insert(map, m*food_line + food_column, "n")
    map = np.array([[map[m*j + i] for i in range(0, m)] for j in range(0, n)])
    
    #carte de la nourriture
    food_quantity = random.randint(5, max_food_quantity)
    food_map = np.array([[0 for i in range(0, m)] for j in range(0, n)])
    food_map = np.delete(food_map, m*food_line + food_column)
    food_map = np.insert(food_map, m*food_line + food_column, food_quantity)
    food_map = np.array([[food_map[m*j + i] for i in range(0, m)] for j in range(0, n)])    
    
    
    
                    #ajout de pièges mortel pour fourmis
    
    for i in range(0,nb_trap):  
        trap_line = random.randint(0, 2*n//3 - 1) 
        trap_column = random.randint(0, 2*m//3 - 1)
        if trap_line > n//3 - 1:
            trap_line = n - 2 - n//3 + trap_line -4
        if trap_column > m//3 - 1: 
            trap_column = m - 2 - m//3 + trap_column -4
        where = random.sample([trap_line, trap_column], 1) 
        if where == trap_line:
            trap_column = random.randint(0, m - 1) 
        else : 
            trap_line = random.randint(0, n - 1) 
        trap = 'p' 
        map = np.delete(map, m*trap_line + trap_column)
        map = np.insert(map, m*trap_line + trap_column, trap)
        map = np.array([[map[m*j + i] for i in range(0, m)] for j in range(0, n) ])
    
    print(map)
    print(food_map)
    return [map, food_map]
print (generate_simple_map(n, m, nb_trap))


[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'p' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'p' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'p' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' 'h' ' ' ' ' ' ' 'p' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' 'h' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'p' ' ' ' ']
 [' ' ' ' ' ' ' ' 'p' ' '

In [36]:
simple_map = generate_simple_map(n, m, nb_trap)
map1 = simple_map[0]
food_map1 = simple_map[1]
map_pher_home1 = np.array([[0 for k in range(0, m)] for l in range(0, n)])
map_pher_food1 = np.array([[0 for k in range(0, m)] for l in range(0, n)])

print(map1)
print(map_pher_home1)
print(map_pher_food1)


def spatial_ants_neighborhood (map, a, b, neigh): #world : List[List[int]], a : int(ligne), b : int(colonne), neigh : int
    
    """Préconditions :0 <= a <= n-1  and 0 <= b <= m-1 and n >= 2*neigh and m >= 2*neigh
    Retourne la matrice des types des voisins de de l'individu en [a, b] avec a la ligne et b la colonne"""
    
    voisin = np.array([[" " for i in range(0, 2*neigh + 1)] for j in range(0, 2*neigh + 1)]) #grille vide de la taille du voisinage
    
    world_without_ind = np.delete(map, m*a + b) #on copie le monde sans l'individu
    world_without_ind = np.insert(world_without_ind, m*a + b, " ") #on remplace la place de l'individu par une case vide (avec un " ")
    world_without_ind = np.array([[world_without_ind[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
    

    if a - neigh >= 0 and a + neigh <= n - 1: # on teste les lignes, il n'y a pas de problème avec les bords (n-1 car les indices commencent à 0)
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[a-neigh + j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(0, 2*neigh + 1)]) 
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[a - neigh + j, i] for i in range(0, b + neigh + 1)] for j in range(0, 2*neigh + 1)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[a - neigh + j, i] for i in range(b - neigh, m)] for j in range(0, 2*neigh + 1)])
    elif a - neigh < 0 : # on atteint le bord haut
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(0, a + neigh + 1)])
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[j, i] for i in range(0, b + neigh + 1)] for j in range(0, a + neigh + 1)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[j, i] for i in range(b - neigh, m)] for j in range(0, a + neigh + 1)])
    else : # a + neigh > n - 1
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(a - neigh, n)]) #on copie 
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[j, i] for i in range(0, b + neigh + 1)] for j in range(a - neigh, n)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[j, i] for i in range(b - neigh, m)] for j in range(a - neigh, n)])
    return voisin

def spatial_pher_neighborhood (map, a, b, neigh): #world : List[List[int]], a : int(ligne), b : int(colonne), neigh : int
    
    """Préconditions :0 <= a <= n-1  and 0 <= b <= m-1 and n >= 2*neigh and m >= 2*neigh
    Retourne la matrice des types des voisins de de l'individu en [a, b] avec a la ligne et b la colonne"""
    
    voisin = np.array([[0 for i in range(0, 2*neigh + 1)] for j in range(0, 2*neigh + 1)]) #grille vide de la taille du voisinage
    
    world_without_ind = np.delete(map, m*a + b) #on copie le monde sans l'individu
    world_without_ind = np.insert(world_without_ind, m*a + b, 0) #on remplace la place de l'individu par une case vide (avec un " ")
    world_without_ind = np.array([[world_without_ind[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
    

    if a - neigh >= 0 and a + neigh <= n - 1: # on teste les lignes, il n'y a pas de problème avec les bords (n-1 car les indices commencent à 0)
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[a-neigh + j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(0, 2*neigh + 1)]) 
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[a - neigh + j, i] for i in range(0, b + neigh + 1)] for j in range(0, 2*neigh + 1)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[a - neigh + j, i] for i in range(b - neigh, m)] for j in range(0, 2*neigh + 1)])
    elif a - neigh < 0 : # on atteint le bord haut
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(0, a + neigh + 1)])
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[j, i] for i in range(0, b + neigh + 1)] for j in range(0, a + neigh + 1)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[j, i] for i in range(b - neigh, m)] for j in range(0, a + neigh + 1)])
    else : # a + neigh > n - 1
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(a - neigh, n)]) #on copie 
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[j, i] for i in range(0, b + neigh + 1)] for j in range(a - neigh, n)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[j, i] for i in range(b - neigh, m)] for j in range(a - neigh, n)])
    return voisin

def ants (map, nb_ants):
    """Precondition : nb_ants >= 0
    Fait apparaitre nb_ants fourmis autour de la fourmilière"""
    ants_map = map
    for k in range(0, n):
        for l in range(0, m):
            if ants_map[k, l] == " ": #on vérifie que la case est vide
                for o in spatial_ants_neighborhood(map, k, l, neigh):
                    for p in o:
                        if p == "h" and nb_ants > 0 : #on regarde si il y a une case fourmilière dans le voisinage
                            if ants_map[k, l] == " ": #on vérifie que la case est vide
                                ants_map = np.delete(ants_map, m*k + l) #on copie le monde en enlevant la case ayant la fourmilière pour voisin
                                ants_map = np.insert(ants_map, m*k + l, "f") #on remplace les cases initialement vide (avec un 0) par une case "fourmi" (avec un 2)
                                ants_map = np.array([[ants_map[m*j + i] for i in range(0, m)] for j in range(0, n) ])
                                nb_ants = nb_ants - 1
    q = 0                            
    while nb_ants > 0: #la fourmilière est entourée de fourmis
        q = q + 1
        for k in range(0, n):
            for l in range(0, m):
                if ants_map[k, l] == " ": #on vérifie que la case est vide
                    for o in spatial_ants_neighborhood(map, k, l, neigh + q):
                        for p in o:
                            if p == "h" and nb_ants > 0 : #on regarde si il y a une case fourmilière dans le voisinage + q
                                if ants_map[k, l] == " ": #on vérifie que la case est vide
                                    ants_map = np.delete(ants_map, m*k + l) #on copie le monde en enlevant la case ayant la fourmilière pour voisin
                                    ants_map = np.insert(ants_map, m*k + l, "f") #on remplace les cases initialement vide (avec un " ") par une case "fourmi" (avec un f)
                                    ants_map = np.array([[ants_map[m*j + i] for i in range(0, m)] for j in range(0, n) ])
                                    nb_ants = nb_ants - 1                        
    return ants_map

def clock_map(map):
    """Renvoie la carte des durées de vie des fourmis"""
    clock = np.array([[0 for i in range(0, m)] for j in range(0, n)])
    for k in range(0, n):
        for l in range(0, m):
            if map[k, l] == "f":
                clock = np.delete(clock, m*k + l)
                clock = np.insert(clock, m*k + l, life_time)
                clock = np.array([[clock[m*o + p] for p in range(0, m)] for o in range(0, n)])
    return clock


def no_pheromone(map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Renvoie True si il n'y a pas des phéromones dans le voisinage de la case et False sinon."""
    for i in spatial_pher_neighborhood(map, a, b, 1):
        for j in i:
            if j != 0 :
                return False
    return True

def max_pheromone(map_pher, a, b):
    """Renvoie la case du voisinage de [a, b] ayant la plus grande valeur de phéromone"""
    maxi = 0
    maxi_line = 0
    maxi_column = 0
    if 0 < a < n-1 and 0 < b < m-1 : #pas de problème avec les bords
        for i in range(0, 3):
            for j in range(0, 3):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j
    if a == 0 and 0 < b < m-1: # on a atteint le bord haut
        for i in range(0, 2):
            for j in range(0, 3):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j
    if a == 0 and b == 0: # on a atteint le bord haut gauche
        for i in range(0, 2):
            for j in range(0, 2):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j      
    if a == 0 and b == m-1: #on a atteint le bord haut droit
        for i in range(0, 2):
            for j in range(0, 2):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j
    if a == n-1 and 0 < b < m-1: # on a atteint le bord bas
        for i in range(0, 2):
            for j in range(0, 3):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j
    if a == n-1 and b == 0: # on a atteint le bord bas gauche
        for i in range(0, 2):
            for j in range(0, 2):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j
    if a == n-1 and b == m-1: #on a atteint le bord bas droit
        for i in range(0, 2):
            for j in range(0, 3):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j
    return [maxi_line, maxi_column]

def no_food(map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Renvoie True si il n'y a pas de nourriture dans le voisinage de la case et False sinon."""
    for i in spatial_ants_neighborhood(map, a, b, 1):
        for j in i:
            if j == "n": #il y a de la nourriture dans le voisinage de la case, on peut utiliser le codage des caractères
                return False
    return True

def choose_the_way(map_pher, a, b):
    """Renvoie 0, 1, 2 ou 3 pour des déplacements vers le haut, la droite, le bas ou la gauche. Choisit le chemin en faisant la moyenne des phéromones
    d'un type autour de la case et en choisissant la deuxième plus élevée (pour choisir le chemin le plus marqué mais dans le bon sens)"""
    env = spatial_pher_neighborhood(map_pher, a, b, 1)
    up = 0 #valeur des phéromones au dessus
    right = 0 #valeur des phéromones à droite
    down = 0 #valeur des phéromones en dessous
    left = 0 #valeur des phéromones à gauche
    next_move = -1
    pher_value = map_pher[a, b]
    print("pher_value :", pher_value)
    if 0 < a < n-1 and 0 < b < m-1 : #pas de problème avec les bords
        up = env[0, 1]
        right = env[1, 2]
        down = env[2, 1]
        left = env[1, 0]
    if a == 0 and 0 < b < m-1: # on a atteint le bord haut
        up = 0
        right = env[0, 2]
        down = env[1, 1]
        left = env[0, 0]     
    if a == 0 and b == 0: # on a atteint le bord haut gauche
        up = 0
        right = env[0, 1]
        down = env[1, 0]
        left = 0
    if a == 0 and b == m-1: #on a atteint le bord haut droit
        up = 0
        right = 0
        down = env[1, 0]
        left = env[0, 0]
    if a == n-1 and 0 < b < m-1: # on a atteint le bord bas
        up = env[0, 1]
        right = env[1, 2]
        down = 0
        left = env[1, 0]
    if a == n-1 and b == 0: # on a atteint le bord bas gauche
        up = env[0, 0]
        right = env[1, 1]
        down = 0
        left = 0
    if a == n-1 and b == m-1: #on a atteint le bord bas droit
        up = env[0, 0]
        right = 0
        down = 0
        left = env[1, 0]
    if  0 < a < n-1 and b == 0: #on atteint le bord gauche
        up = env[0, 0]
        right = env[1, 1]
        down = env[2, 0]
        left = 0
    if  0 < a < n-1 and b == m-1: #on atteint le bord droit
        up = env[0, 1]
        right = 0
        down = env[2, 1]
        left = env[1, 0]
    print("up", up)
    print("right", right)
    print("down", down)
    print("left", left)
    
    #on choisit la deuxième direction la plus chargée en phéromones
    list_pher = [up, down, right, left]
    second = sorted(list_pher)[2]
    print("second",second)
    if pher_value == 0: #la fourmi est à côté d'une trace de phéromones
        if second == 0: #il n'y a qu'une direction possible pour les phéromones
            print("po")
            second = max(list_pher)
            if second == up:
                next_move = 0
            if second == right:
                next_move = 1
            if second == down:
                next_move = 2
            if second == left:
                next_move = 3
        else : #on prend la deuxième direction la plus forte
            print("re")
            if second == up:
                next_move = 0
            if second == right:
                next_move = 1
            if second == down:
                next_move = 2
            if second == left:
                next_move = 3
    else: #la fourmi est sur une trace de phéromones
        if up == pher_value - 1:
            return 0
            print("cor")
        if right == pher_value - 1:
            return 1
            print("cor")
        if down == pher_value - 1:
            return 2
            print("cor")
        if left == pher_value - 1:
            return 3
            print("cor")
        else:
            if second == 0: #il n'y a qu'une direction possible pour les phéromones
                print("po2")
                second = max(list_pher)
                if second == up:
                    next_move = 0
                if second == right:
                    next_move = 1
                if second == down:
                    next_move = 2
                if second == left:
                    next_move = 3
            else : #on prend la deuxième direction la plus forte
                print("re2")
                if second == up:
                    next_move = 0
                if second == right:
                    next_move = 1
                if second == down:
                    next_move = 2
                if second == left:
                    next_move = 3
    print(next_move)
    return next_move

#print(np.array([[0, 0, 6, 7, 8, 0, 4, 7, 7, 0, 8, 3, 9, 7, 8, 3, 5, 5] for i in range(0, n)]))
#print(choose_the_way(np.array([[0, 0, 6, 7, 8, 0, 4, 7, 7, 0, 8, 3, 9, 7, 8, 3, 5, 5] for i in range(0, n)]), 3, 1))
    
def spatial_relocation (map, map_pher_home, map_pher_food,clock_map, a, b): #map : List[List[int]], a : int(ligne), b : int(colonne)
    """Préconditions : 0 <= a <= n-1  and 0 <= b <= m-1  and 0 <= c <= n-1  and 0 <= d <= m-1 and n >= 2*neigh and m >= 2*neigh and map[c,d] == 0
    Renvoie le monde dans lequel l'individu de coordonnées [a, b] a été déplacé aléatoirement si pas de phéromone, suit les phéromone_food
    si la fourmi est en quête, suit les phéromones_home si la fourmi a de la nourriture. Les matrices sont également modifiées en conséquent."""
    ant = map[a, b] #on enregistre la cellule à déplacer (une fourmi)
    pher_food_map = map_pher_food
    pher_home_map = map_pher_home
    relocation = map
    clock = clock_map
    move_line = a
    move_column = b
    if ant == "f": #déplacement des fourmis en recherche de nourriture
        if pher_home_map[move_line, move_column] == 0: # on ne pose pas de phéromones si il y a déja une trace
            pher_home_map = np.delete(pher_home_map, m*move_line + move_column)
            pher_home_map = np.insert(pher_home_map, m*move_line + move_column, pers_pher) # sur la map de phéromones "maison" la case de même coordonnées que la fourmi prend la valeur de la persistance des phéromones
            pher_home_map = np.array([[pher_home_map[m*j + i] for i in range(0, m)] for j in range(0, n)])
        if no_pheromone(map_pher_food, move_line, move_column): #il n'y a pas de phéromone "nourriture" à coté de la fourmi
            next_move = random.randint(0, 3) #on décide le prochain mouvement au hasard
            if next_move == 0:
                if move_line > 0 and (relocation[move_line - 1, move_column] == " " or relocation[move_line - 1, move_column] == "p") and pher_home_map[move_line - 1, move_column] < pers_pher - memory: #la prochaine case existe et est vide
                    move_line = move_line - 1
                    move_column = move_column
                if move_line > 1 and (relocation[move_line - 2, move_column] == " " or relocation[move_line - 2, move_column] == "p") and (relocation[move_line - 1, move_column] == "f" or relocation[move_line - 1, move_column] == "s") and pher_home_map[move_line - 2, move_column] < pers_pher - memory:
                    move_line = move_line - 2
                    move_column = move_column
            elif next_move == 1:
                if move_column < m - 1 and (relocation[move_line, move_column + 1] == " " or relocation[move_line, move_column + 1] == "p") and pher_home_map[move_line, move_column + 1] < pers_pher - memory:
                    move_line = move_line
                    move_column = move_column + 1
                if move_column < m - 2 and (relocation[move_line, move_column + 2] == " " or relocation[move_line, move_column + 2] == "p") and (relocation[move_line, move_column + 1] == "f" or relocation[move_line, move_column + 1] == "s") and pher_home_map[move_line, move_column + 2] < pers_pher - memory:
                    move_line = move_line
                    move_column = move_column + 2
            elif next_move == 2:
                if move_line < n - 1 and (relocation[move_line + 1, move_column] == " " or relocation[move_line + 1, move_column] == "p") and pher_home_map[move_line + 1, move_column] < pers_pher - memory:
                    move_line = move_line + 1
                    move_column = move_column
                if move_line < n - 2 and (relocation[move_line + 2, move_column] == " " or relocation[move_line + 2, move_column] == "p") and (relocation[move_line + 1, move_column] == "f" or relocation[move_line + 1, move_column] == "s") and pher_home_map[move_line + 2, move_column] < pers_pher - memory:
                    move_line = move_line + 2
                    move_column = move_column
            elif next_move == 3:
                if move_column > 0 and (relocation[move_line, move_column - 1] == " " or relocation[move_line, move_column - 1] == "p") and pher_home_map[move_line, move_column - 1] < pers_pher - memory:
                    move_line = move_line
                    move_column = move_column - 1
                if move_column > 1 and (relocation[move_line, move_column - 2] == " " or relocation[move_line, move_column - 2] == "p") and (relocation[move_line, move_column - 1] == "f" or relocation[move_line, move_column - 1] == "s") and pher_home_map[move_line, move_column - 2] < pers_pher - memory:
                    move_line = move_line
                    move_column = move_column - 2
                    
            #on déplace et modifie l'horloge de la fourmi
            clock1 = np.array(clock)
            clock = np.delete(clock1, m*move_line + move_column)
            clock = np.insert(clock, m*move_line + move_column, int(clock1[a, b] - 1))
            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
            if a != move_line or b != move_column:
                clock = np.delete(clock, m*a + b)
                clock = np.insert(clock, m*a + b, 0)
                clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])   
                
             #mort d'une fourmi si déplacement sur piège
            if relocation[move_line, move_column] == "p":
                clock = np.delete(clock, m*move_line + move_column)
                clock = np.insert(clock, m*move_line + move_column, 0)
                clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])   
                
            # on supprime l'indidu du monde
            relocation = np.delete(relocation, m*a + b) #on copie le monde sans l'individu
            relocation = np.insert(relocation, m*a + b, " ") #on remplace la place de l'individu par une case vide (avec un " ")
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                # on remplace la case vide par l'individu
            relocation = np.delete(relocation, m*(move_line) + move_column) #on copie le monde sans la case vide
            relocation = np.insert(relocation, m*(move_line) + move_column, "g") #on remplace la place de la case vide par l'individu (ant)
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau


 
        else : #l'individu suit le chemin de phéromones le plus marqué
            relocation = np.delete(relocation, m*move_line + move_column) #on copie le monde sans l'individu
            relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n)]) #on remet le monde sous forme de tableau
            way_food = choose_the_way(pher_food_map, move_line, move_column)
            
            if way_food == 0:
                if move_line > 0 and (relocation[move_line - 1, move_column] == " " or relocation[move_line - 1, move_column] == "p"): #la prochaine case existe et est vide
                    move_line = move_line - 1
                    move_column = move_column
                if move_line > 1 and (relocation[move_line - 2, move_column] == " " or relocation[move_line - 2, move_column] == "p") and (relocation[move_line - 1, move_column] == "f" or relocation[move_line - 1, move_column] == "s"): #la prochaine case existe et est vide
                    move_line = move_line - 2
                    move_column = move_column
                    
            if way_food == 1:
                if move_column < m - 1 and (relocation[move_line, move_column + 1] == " " or relocation[move_line + 1, move_column + 1] == "p"):
                    move_line = move_line
                    move_column = move_column + 1
                if move_column < m - 2 and (relocation[move_line, move_column + 2] == " " or relocation[move_line, move_column + 2] == "p") and (relocation[move_line, move_column + 1] == "f" or relocation[move_line, move_column + 1] == "s"):
                    move_line = move_line
                    move_column = move_column + 2
                    
            if way_food == 2:
                if move_line < n - 1 and (relocation[move_line + 1, move_column] == " " or relocation[move_line + 1, move_column] == "p"):
                    move_line = move_line + 1
                    move_column = move_column
                if move_line < n - 2 and (relocation[move_line + 2, move_column] == " " or relocation[move_line + 2, move_column] == "p") and (relocation[move_line + 1, move_column] == "f" or relocation[move_line + 1, move_column] == "s"):
                    move_line = move_line + 2
                    move_column = move_column

            if way_food == 3:
                if move_column > 0 and (relocation[move_line, move_column - 1] == " " or relocation[move_line, move_column - 1] == "p"):
                    move_line = move_line
                    move_column = move_column - 1
                if move_column > 1 and (relocation[move_line, move_column - 2] == " " or relocation[move_line, move_column - 2] == "p") and (relocation[move_line, move_column - 1] == "f" or relocation[move_line, move_column - 1] == "s"):
                    move_line = move_line
                    move_column = move_column - 2
                    

            #on déplace et modifie l'horloge de la fourmi
            clock1 = np.array(clock)
            clock = np.delete(clock1, m*move_line + move_column)
            clock = np.insert(clock, m*move_line + move_column, int(clock1[a, b] - 1))
            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
            if a != move_line or b != move_column:
                clock = np.delete(clock, m*a + b)
                clock = np.insert(clock, m*a + b, 0)
                clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)]) 
                
             #mort d'une fourmi si déplacement sur piège
            if relocation[move_line, move_column] == "p":
                clock = np.delete(clock, m*move_line + move_column)
                clock = np.insert(clock, m*move_line + move_column, 0)
                clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])      
                
            # on supprime l'indidu du monde
            relocation = np.delete(relocation, m*a + b) #on copie le monde sans l'individu
            relocation = np.insert(relocation, m*a + b, " ") #on remplace la place de l'individu par une case vide (avec un " ")
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                # on remplace la case vide par l'individu
            relocation = np.delete(relocation, m*(move_line) + move_column) #on copie le monde sans la case vide
            relocation = np.insert(relocation, m*(move_line) + move_column, "g") #on remplace la place de la case vide par l'individu (ant)
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau


                
        if pher_home_map[move_line, move_column] == 0: # on ne pose pas de phéromones si il y a déja une trace
            pher_home_map = np.delete(pher_home_map, m*move_line + move_column)
            pher_home_map = np.insert(pher_home_map, m*move_line + move_column, pers_pher - 1) # sur la map de phéromones "maison" la case de même coordonnées que la fourmi prend la valeur de la persistance des phéromones
            pher_home_map = np.array([[pher_home_map[m*j + i] for i in range(0, m)] for j in range(0, n)])
            
        #la fourmi est morte
        if clock[move_line, move_column] <= 0:
            relocation = np.delete(relocation, m*move_line + move_column)
            relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
        
    if ant == "s": #déplacement des fourmis ayant trouvé de la nourriture
        pher_food_map = np.delete(pher_food_map, m*move_line + move_column)
        pher_food_map = np.insert(pher_food_map, m*move_line + move_column, pher_food_map[m*a + b] + pers_pher) # sur la map de phéromones "nourriture" la case de même coordonnées que la fourmi prend la valeur de la persistance des phéromones
        pher_food_map = np.array([[pher_food_map[m*j + i] for i in range(0, m)] for j in range(0, n)])
        if no_pheromone(map_pher_home, move_line, move_column): #il n'y a pas de phéromone "maison" à coté de la fourmi
            next_move = random.randint(0, 3) #on décide le prochain mouvement au hasard
            if next_move == 0:
                if move_line > 0 and (relocation[move_line - 1, move_column] == " " or relocation[move_line - 1, move_column] == "p") and pher_food_map[move_line - 1, move_column] % pers_pher < pers_pher - memory: #la prochaine case existe et est vide
                    move_line = move_line - 1
                    move_column = move_column
                if move_line > 1 and (relocation[move_line - 2, move_column] == " " or relocation[move_line - 2, move_column] == "p") and (relocation[move_line - 1, move_column] == "f" or relocation[move_line - 1, move_column] == "s") and pher_food_map[move_line - 2, move_column] < pers_pher - memory:
                    move_line = move_line - 2
                    move_column = move_column
            elif next_move == 1:
                if move_column < m - 1 and (relocation[move_line, move_column + 1] == " " or relocation[move_line, move_column + 1] == "p") and pher_food_map[move_line, move_column + 1] % pers_pher < pers_pher - memory:
                    move_line = move_line
                    move_column = move_column + 1
                if move_column < m - 2 and (relocation[move_line, move_column + 2] == " " or relocation[move_line, move_column + 2] == "p") and (relocation[move_line, move_column + 1] == "f" or relocation[move_line, move_column + 1] == "s") and pher_food_map[move_line, move_column + 2] < pers_pher - memory:
                    move_line = move_line
                    move_column = move_column + 2
            elif next_move == 2:
                if move_line < n - 1 and (relocation[move_line + 1, move_column] == " " or relocation[move_line + 1, move_column] == "p") and pher_food_map[move_line + 1, move_column] % pers_pher < pers_pher - memory:
                    move_line = move_line + 1
                    move_column = move_column
                if move_line < n - 2 and (relocation[move_line + 2, move_column] == " " or relocation[move_line + 2, move_column] == "p") and (relocation[move_line + 1, move_column] == "f" or relocation[move_line + 1, move_column] == "s") and pher_food_map[move_line + 2, move_column] < pers_pher - memory:
                    move_line = move_line + 2
                    move_column = move_column
            elif next_move == 3:
                if move_column > 0 and (relocation[move_line, move_column - 1] == " " or relocation[move_line, move_column - 1] == "p") and pher_food_map[move_line, move_column - 1] % pers_pher < pers_pher - memory:
                    move_line = move_line
                    move_column = move_column - 1
                if move_column > 1 and (relocation[move_line, move_column - 2] == " " or relocation[move_line, move_column - 2] == "p") and (relocation[move_line, move_column - 1] == "f" or relocation[move_line, move_column - 1] == "s") and pher_food_map[move_line, move_column - 2] < pers_pher - memory:
                    move_line = move_line
                    move_column = move_column - 2
                    
                                        
            #on déplace et modifie l'horloge de la fourmi
            clock1 = np.array(clock)
            clock = np.delete(clock1, m*move_line + move_column)
            clock = np.insert(clock, m*move_line + move_column, int(clock1[a, b] - 1))
            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
            if a != move_line or b != move_column:
                clock = np.delete(clock, m*a + b)
                clock = np.insert(clock, m*a + b, 0)
                clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)]) 
                
             #mort d'une fourmi si déplacement sur piège
            if relocation[move_line, move_column] == "p":
                clock = np.delete(clock, m*move_line + move_column)
                clock = np.insert(clock, m*move_line + move_column, 0)
                clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])     
                
            # on supprime l'indidu du monde
            relocation = np.delete(relocation, m*a + b) #on copie le monde sans l'individu
            relocation = np.insert(relocation, m*a + b, " ") #on remplace la place de l'individu par une case vide (avec un " ")
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                # on remplace la case vide par l'individu
            relocation = np.delete(relocation, m*(move_line) + move_column) #on copie le monde sans la case vide
            relocation = np.insert(relocation, m*(move_line) + move_column, "t") #on remplace la place de la case vide par l'individu (ant)
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau


                    
        else : #l'individu suit le chemin de phéromones le plus marqué
            print("oui")
            relocation = np.delete(relocation, m*move_line + move_column) #on copie le monde sans l'individu
            relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n)]) #on remet le monde sous forme de tableau
            way_home = choose_the_way(pher_home_map, move_line, move_column)
            if way_home == 0:
                if move_line > 0 and (relocation[move_line - 1, move_column] == " " or relocation[move_line - 1, move_column] == "p"): #la prochaine case existe et est vide
                    move_line = move_line - 1
                    move_column = move_column
                if move_line > 1 and (relocation[move_line - 2, move_column] == " " or relocation[move_line - 2, move_column] == "p") and (relocation[move_line - 1, move_column] == "f" or relocation[move_line - 1, move_column] == "s"): #la prochaine case existe et est vide
                    move_line = move_line - 2
                    move_column = move_column
                    
            if way_home == 1:
                if move_column < m - 1 and (relocation[move_line, move_column + 1] == " " or relocation[move_line + 1, move_column + 1] == "p"):
                    move_line = move_line
                    move_column = move_column + 1
                if move_column < m - 2 and (relocation[move_line, move_column + 2] == " " or relocation[move_line, move_column + 2] == "p") and (relocation[move_line, move_column + 1] == "f" or relocation[move_line, move_column + 1] == "s"):
                    move_line = move_line
                    move_column = move_column + 2
                    
            if way_home == 2:
                if move_line < n - 1 and (relocation[move_line + 1, move_column] == " " or relocation[move_line + 1, move_column] == "p"):
                    move_line = move_line + 1
                    move_column = move_column
                if move_line < n - 2 and (relocation[move_line + 2, move_column] == " " or relocation[move_line + 2, move_column] == "p") and (relocation[move_line + 1, move_column] == "f" or relocation[move_line + 1, move_column] == "s"):
                    move_line = move_line + 2
                    move_column = move_column

            if way_home == 3:
                if move_column > 0 and (relocation[move_line, move_column - 1] == " " or relocation[move_line, move_column - 1] == "p"):
                    move_line = move_line
                    move_column = move_column - 1
                if move_column > 1 and (relocation[move_line, move_column - 2] == " " or relocation[move_line, move_column - 2] == "p") and (relocation[move_line, move_column - 1] == "f" or relocation[move_line, move_column - 1] == "s"):
                    move_line = move_line
                    move_column = move_column - 2
                    
                    
            #on déplace et modifie l'horloge de la fourmi
            clock1 = np.array(clock)
            clock = np.delete(clock1, m*move_line + move_column)
            clock = np.insert(clock, m*move_line + move_column, int(clock1[a, b] - 1))
            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
            if a != move_line or b != move_column:
                clock = np.delete(clock, m*a + b)
                clock = np.insert(clock, m*a + b, 0)
                clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])   
                
             #mort d'une fourmi si déplacement sur piège
            if relocation[move_line, move_column] == "p":
                clock = np.delete(clock, m*move_line + move_column)
                clock = np.insert(clock, m*move_line + move_column, 0)
                clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)]) 
                
            # on supprime l'indidu du monde
            relocation = np.delete(relocation, m*a + b) #on copie le monde sans l'individu
            relocation = np.insert(relocation, m*a + b, " ") #on remplace la place de l'individu par une case vide (avec un " ")
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                # on remplace la case vide par l'individu
            relocation = np.delete(relocation, m*(move_line) + move_column) #on copie le monde sans la case vide
            relocation = np.insert(relocation, m*(move_line) + move_column, "t") #on remplace la place de la case vide par l'individu (ant)
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
            
        
        #la fourmi est morte
        if clock[move_line, move_column] <= 0:
            relocation = np.delete(relocation, m*move_line + move_column)
            relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau

    return [relocation, pher_home_map, pher_food_map, clock]

def take_food(map, food_map, clock_map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Renvoie les mêmes cartes avec une unité de moins sur la nourriture, un bonus de temps de vie sur l'horloge et une fourmi transformée"""
    mapf = map
    food = food_map
    clock = clock_map
    food_value = 0
    life = clock[a, b]
    print("life", life)
    if map[a, b] == "g": # la cellule sélectionnée est une fourmi sans nourriture ayant bougé
        if 0 < a < n-1 and 0 < b < m-1 : #pas de problème avec les bords
            for i in range(0, 3):
                for j in range(0, 3):
                    if not(no_food(map, a, b)):
                        if spatial_pher_neighborhood(food, a, b, neigh)[i, j] != 0: #la case a de la nourriture
                            food_value = spatial_pher_neighborhood(food, a, b, neigh)[i, j]
                            food = np.delete(food, m*(a - 1 + i) + (b - 1 + j))
                            food = np.insert(food, m*(a - 1 + i) + (b - 1 + j), food_value - 1) #on enlève 1 à la valeur de nourriture
                            food = np.array([[food[m*k + l] for l in range(0, m)] for k in range(0, n)])
                            
                            clock = np.delete(clock, m*a + b)
                            clock = np.insert(clock, m*a + b, int(life + reward))
                            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
                            mapf = np.delete(mapf, m*a + b) #on copie le monde sans l'individu
                            mapf = np.insert(mapf, m*a + b, "t") #on remplace la place de l'individu par une fourmi avec nourriture mais s'étant déplacée (un "t")
                            mapf = np.array([[mapf[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                            # on enlève la nourriture si sa valeur est 0
                            if food_value - 1 == 0:
                                mapf = np.delete(mapf, m*(a - 1 + i) + (b - 1 + j))
                                mapf = np.insert(mapf, m*(a - 1 + i) + (b - 1 + j), " ")
                                mapf = np.array([[mapf[m*k + l] for l in range(0, m)] for k in range(0, n)])
        if a == 0 and 0 < b < m-1: # on a atteint le bord haut
            for i in range(0, 2):
                for j in range(0, 3):
                    if not(no_food(map, a, b)):
                        if spatial_pher_neighborhood(food, a, b, neigh)[i, j] != 0: #la case a de la nourriture
                            food_value = spatial_pher_neighborhood(food, a, b, neigh)[i, j]
                            food = np.delete(food, m*(a + i) + (b - 1 + j))
                            food = np.insert(food, m*(a + i) + (b - 1 + j), food_value - 1) #on enlève 1 à la valeur de nourriture
                            food = np.array([[food[m*k + l] for l in range(0, m)] for k in range(0, n)])
                            
                            clock = np.delete(clock, m*a + b)
                            clock = np.insert(clock, m*a + b, int(life + reward))
                            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
                            mapf = np.delete(mapf, m*a + b) #on copie le monde sans l'individu
                            mapf = np.insert(mapf, m*a + b, "t") #on remplace la place de l'individu par une fourmi avec nourriture mais s'étant déplacée (un "t")
                            mapf = np.array([[mapf[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                            # on enlève la nourriture si sa valeur est 0
                            if food_value - 1 == 0:
                                mapf = np.delete(mapf, m*(a + i) + (b - 1 + j))
                                mapf = np.insert(mapf, m*(a + i) + (b - 1 + j), " ")
                                mapf = np.array([[mapf[m*k + l] for l in range(0, m)] for k in range(0, n)])
        if a == 0 and b == 0: # on a atteint le bord haut gauche
            for i in range(0, 2):
                for j in range(0, 2):
                    if not(no_food(map, a, b)):
                        if spatial_pher_neighborhood(food, a, b, neigh)[i, j] != 0: #la case a de la nourriture
                            food_value = spatial_pher_neighborhood(food, a, b, neigh)[i, j]
                            food = np.delete(food, m*(a + i) + (b + j))
                            food = np.insert(food, m*(a + i) + (b + j), food_value - 1) #on enlève 1 à la valeur de nourriture
                            food = np.array([[food[m*k + l] for l in range(0, m)] for k in range(0, n)])
                            
                            clock = np.delete(clock, m*a + b)
                            clock = np.insert(clock, m*a + b, int(life + reward))
                            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
                            mapf = np.delete(mapf, m*a + b) #on copie le monde sans l'individu
                            mapf = np.insert(mapf, m*a + b, "t") #on remplace la place de l'individu par une fourmi avec nourriture mais s'étant déplacée (un "t")
                            mapf = np.array([[mapf[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                            # on enlève la nourriture si sa valeur est 0
                            if food_value - 1 == 0:
                                mapf = np.delete(mapf, m*(a + i) + (b + j))
                                mapf = np.insert(mapf, m*(a + i) + (b + j), " ")
                                mapf = np.array([[mapf[m*k + l] for l in range(0, m)] for k in range(0, n)])
        if a == 0 and b == m-1: #on a atteint le bord haut droit
            for i in range(0, 2):
                for j in range(0, 2):
                    if not(no_food(map, a, b)):
                        if spatial_pher_neighborhood(food, a, b, neigh)[i, j] != 0: #la case a de la nourriture
                            food_value = spatial_pher_neighborhood(food, a, b, neigh)[i, j]
                            food = np.delete(food, m*(a + i) + (b - 1 + j))
                            food = np.insert(food, m*(a + i) + (b - 1 + j), food_value - 1) #on enlève 1 à la valeur de nourriture
                            food = np.array([[food[m*k + l] for l in range(0, m)] for k in range(0, n)])
                            
                            clock = np.delete(clock, m*a + b)
                            clock = np.insert(clock, m*a + b, int(life + reward))
                            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
                            mapf = np.delete(mapf, m*a + b) #on copie le monde sans l'individu
                            mapf = np.insert(mapf, m*a + b, "t") #on remplace la place de l'individu par une fourmi avec nourriture mais s'étant déplacée (un "t")
                            mapf = np.array([[mapf[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                            # on enlève la nourriture si sa valeur est 0
                            if food_value - 1 == 0:
                                mapf = np.delete(mapf, m*(a + i) + (b - 1 + j))
                                mapf = np.insert(mapf, m*(a + i) + (b - 1 + j), " ")
                                mapf = np.array([[mapf[m*k + l] for l in range(0, m)] for k in range(0, n)])
        if a == n-1 and 0 < b < m-1: # on a atteint le bord bas
            for i in range(0, 2):
                for j in range(0, 3):
                    if not(no_food(map, a, b)):
                        if spatial_pher_neighborhood(food, a, b, neigh)[i, j] != 0: #la case a de la nourriture
                            food_value = spatial_pher_neighborhood(food, a, b, neigh)[i, j]
                            food = np.delete(food, m*(a - 1 + i) + (b - 1 + j))
                            food = np.insert(food, m*(a - 1 + i) + (b - 1 + j), food_value - 1) #on enlève 1 à la valeur de nourriture
                            food = np.array([[food[m*k + l] for l in range(0, m)] for k in range(0, n)])
                            
                            clock = np.delete(clock, m*a + b)
                            clock = np.insert(clock, m*a + b, int(life + reward))
                            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
                            mapf = np.delete(mapf, m*a + b) #on copie le monde sans l'individu
                            mapf = np.insert(mapf, m*a + b, "t") #on remplace la place de l'individu par une fourmi avec nourriture mais s'étant déplacée (un "t")
                            mapf = np.array([[mapf[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                            # on enlève la nourriture si sa valeur est 0
                            if food_value - 1 == 0:
                                mapf = np.delete(mapf, m*(a - 1 + i) + (b - 1 + j))
                                mapf = np.insert(mapf, m*(a - 1 + i) + (b - 1 + j), " ")
                                mapf = np.array([[mapf[m*k + l] for l in range(0, m)] for k in range(0, n)])
        if a == n-1 and b == 0: # on a atteint le bord bas gauche
            for i in range(0, 2):
                for j in range(0, 2):
                    if not(no_food(map, a, b)):
                        if spatial_pher_neighborhood(food, a, b, neigh)[i, j] != 0: #la case a de la nourriture
                            food_value = spatial_pher_neighborhood(food, a, b, neigh)[i, j]
                            food = np.delete(food, m*(a - 1 + i) + (b + j))
                            food = np.insert(food, m*(a - 1 + i) + (b + j), food_value - 1) #on enlève 1 à la valeur de nourriture
                            food = np.array([[food[m*k + l] for l in range(0, m)] for k in range(0, n)])
                            
                            clock = np.delete(clock, m*a + b)
                            clock = np.insert(clock, m*a + b, int(life + reward))
                            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
                            mapf = np.delete(mapf, m*a + b) #on copie le monde sans l'individu
                            mapf = np.insert(mapf, m*a + b, "t") #on remplace la place de l'individu par une fourmi avec nourriture mais s'étant déplacée (un "t")
                            mapf = np.array([[mapf[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                            # on enlève la nourriture si sa valeur est 0
                            if food_value - 1 == 0:
                                mapf = np.delete(mapf, m*(a - 1 + i) + (b + j))
                                mapf = np.insert(mapf, m*(a - 1 + i) + (b + j), " ")
                                mapf = np.array([[mapf[m*k + l] for l in range(0, m)] for k in range(0, n)])
        if a == n-1 and b == m-1: #on a atteint le bord bas droit
            for i in range(0, 2):
                for j in range(0, 2):
                    if not(no_food(map, a, b)):
                        if spatial_pher_neighborhood(food, a, b, neigh)[i, j] != 0: #la case a de la nourriture
                            food_value = spatial_pher_neighborhood(food, a, b, neigh)[i, j]
                            food = np.delete(food, m*(a - 1 + i) + (b - 1 + j))
                            food = np.insert(food, m*(a - 1 + i) + (b - 1 + j), food_value - 1) #on enlève 1 à la valeur de nourriture
                            food = np.array([[food[m*k + l] for l in range(0, m)] for k in range(0, n)])
                            
                            clock = np.delete(clock, m*a + b)
                            clock = np.insert(clock, m*a + b, int(life + reward))
                            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
                            mapf = np.delete(mapf, m*a + b) #on copie le monde sans l'individu
                            mapf = np.insert(mapf, m*a + b, "t") #on remplace la place de l'individu par une fourmi avec nourriture mais s'étant déplacée (un "t")
                            mapf = np.array([[mapf[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                            # on enlève la nourriture si sa valeur est 0
                            if food_value - 1 == 0:
                                mapf = np.delete(mapf, m*(a - 1 + i) + (b - 1 + j))
                                mapf = np.insert(mapf, m*(a - 1 + i) + (b - 1 + j), " ")
                                mapf = np.array([[mapf[m*k + l] for l in range(0, m)] for k in range(0, n)])

    return [mapf, food, clock]
    
def no_home(map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Renvoie True si il n'y a pas de case fourmilière dans le voisinage et False sinon."""
    if map[a, b] == "t":
        for i in spatial_ants_neighborhood(map, a, b, 1):
            for j in i:
                if j == "h": #il n'y a pas de case fourmilière à côté
                    return False
    return True

def drop_food(map, food_count, a, b):
    """Retourne la map où la fourmi avec nourriture en [a, b] dépose sa nourriture dans le nid et redevient chercheuse, ajoute un au compteur de nourriture"""
    ant = map[a, b]
    map1 = map
    count = food_count
    if ant == "t": #la case est bien une fourmi avec nourriture ayant bougé
        if not(no_home(map, a, b)):
            map1 = np.delete(map1, m*a + b)
            map1 = np.insert(map1, m*a + b, "g")
            map1 = np.array([[map1[m*k+l] for l in range(0, m)] for k in range(0, n)])
            count = count + 1
    return [map1, count]
    
def one_turn(map, map_pher_home, map_pher_food, food_map, clock_map, count):
    """Renvoie la map après avoir effectuer sur toutes les cases une relocation"""
    one_map = map
    one_pher_home_map = map_pher_home
    one_pher_food_map = map_pher_food
    one_food_map = food_map
    one_clock = clock_map
    one_count = count
    for i in range(0, n):
        for j in range(0, m):
            if one_map[i, j] != " ":
                a = spatial_relocation(one_map, one_pher_home_map, one_pher_food_map, one_clock, i, j)
                one_map = a[0]
                one_pher_home_map = a[1]
                one_pher_food_map = a[2]
                one_clock = a[3]
    for i in range(0, n):
        for j in range(0, m):
            if one_map[i, j] == "g" or one_map[i, j] == "t":
                b = take_food(one_map, one_food_map, one_clock, i, j) #si une fourmi est à côté de nourriture on fait la procédure
                one_map = b[0]
                c = drop_food(one_map, one_count, i, j)
                one_map = c[0] #si une fourmi avec nourriture arrive au nid elle dépose la nourriture
                one_food_map = b[1]
                one_clock = b[2]
                one_count = c[1]

    for i in range(0, n):
        for j in range(0, m):
            if one_map[i, j] == "g":
                one_map = np.delete(one_map, m*i + j)
                one_map = np.insert(one_map, m*i + j, "f")
                one_map = np.array([[one_map[m*k + l] for l in range(0, m)] for k in range(0, n)])
            elif one_map[i, j] == "t":
                one_map = np.delete(one_map, m*i + j)
                one_map = np.insert(one_map, m*i + j, "s")
                one_map = np.array([[one_map[m*k + l] for l in range(0, m)] for k in range(0, n)])
                
    #création d'une fourmi si le compte de nourriture atteint creation_rate
    if one_count >= creation_rate:
        for i in range(1, n - 1):
            for j in range(1, m - 1):
                for k in range(0, 3):
                    for l in range(0, 3):
                        if spatial_ants_neighborhood(one_map, i, j, neigh)[k, l] == "h" and one_map[i, j] == " " and one_count >= creation_rate:
                            one_map = np.delete(one_map, m*i + j)
                            one_map = np.insert(one_map, m*i + j, "f")
                            one_map = np.array([[one_map[m*k + l] for l in range(0, m)] for k in range(0, n)])
                            one_clock = np.delete(one_clock, m*i + j)
                            one_clock = np.insert(one_clock, m*i + j, life_time)
                            one_clock = np.array([[one_clock[m*k + l] for l in range(0, m)] for k in range(0, n)])                            
                            one_count = one_count - creation_rate
    one = [one_map, one_pher_home_map, one_pher_food_map, one_food_map, one_clock, one_count]

    return one
#print(one_turn(ants(map1, nb_ants), map_pher_home1, map_pher_food1, food_map1, clock_map(ants(map1, nb_ants)), food_count))
    
def one_eratic_move(map, map_pher_home, map_pher_food, clock_map, a, b):
    """Retourne les différentes maps après un déplacement aléatoire de la fourmis, y compris si elle suit un chemin de phéromones"""
    ant = map[a, b] #on enregistre la cellule à déplacer (une fourmi)
    pher_food_map = map_pher_food
    pher_home_map = map_pher_home
    relocation = map
    clock = clock_map
    move_line = a
    move_column = b
    if ant == "f": #déplacement des fourmis en recherche de nourriture
        pher_home_map = np.delete(pher_home_map, m*move_line + move_column)
        pher_home_map = np.insert(pher_home_map, m*move_line + move_column, pers_pher) # sur la map de phéromones "maison" la case de même coordonnées que la fourmi prend la valeur de la persistance des phéromones
        pher_home_map = np.array([[pher_home_map[m*j + i] for i in range(0, m)] for j in range(0, n)])
        next_move = random.randint(0, 3) #on décide le prochain mouvement au hasard
        if next_move == 0:
             if move_line > 0 and relocation[move_line - 1, move_column] == " " and pher_home_map[move_line - 1, move_column] < pers_pher - memory: #la prochaine case existe et est vide
                move_line = move_line - 1
                move_column = move_column
                    
        elif next_move == 1:
            if move_column < m - 1 and relocation[move_line, move_column + 1] == " " and pher_home_map[move_line, move_column + 1] < pers_pher - memory:
                move_line = move_line
                move_column = move_column + 1
                    
        elif next_move == 2:
            if move_line < n - 1 and relocation[move_line + 1, move_column] == " " and pher_home_map[move_line + 1, move_column] < pers_pher - memory:
                move_line = move_line + 1
                move_column = move_column
                    
        elif next_move == 3:
            if move_column > 0 and relocation[move_line, move_column - 1] == " " and pher_home_map[move_line, move_column - 1] < pers_pher - memory:
                move_line = move_line
                move_column = move_column - 1
                    
            #on déplace et modifie l'horloge de la fourmi
        clock1 = np.array(clock)
        clock = np.delete(clock1, m*move_line + move_column)
        clock = np.insert(clock, m*move_line + move_column, int(clock1[a, b] - 1))
        clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
        if a != move_line or b != move_column:
            clock = np.delete(clock, m*a + b)
            clock = np.insert(clock, m*a + b, 0)
            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])     
                
            # on supprime l'indidu du monde
        relocation = np.delete(relocation, m*a + b) #on copie le monde sans l'individu
        relocation = np.insert(relocation, m*a + b, " ") #on remplace la place de l'individu par une case vide (avec un " ")
        relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                # on remplace la case vide par l'individu
        relocation = np.delete(relocation, m*(move_line) + move_column) #on copie le monde sans la case vide
        relocation = np.insert(relocation, m*(move_line) + move_column, "g") #on remplace la place de la case vide par l'individu (ant)
        relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
    if ant == "s": #déplacement des fourmis ayant trouvé de la nourriture
        pher_food_map = np.delete(pher_food_map, m*move_line + move_column)
        pher_food_map = np.insert(pher_food_map, m*move_line + move_column, pers_pher) # sur la map de phéromones "nourriture" la case de même coordonnées que la fourmi prend la valeur de la persistance des phéromones
        pher_food_map = np.array([[pher_food_map[m*j + i] for i in range(0, m)] for j in range(0, n)])
        next_move = random.randint(0, 3) #on décide le prochain mouvement au hasard
        if next_move == 0:
            if move_line > 0 and relocation[move_line - 1, move_column] == " " and pher_food_map[move_line, move_column - 1] < pers_pher - 2: #la prochaine case existe et est vide
                move_line = move_line - 1
                move_column = move_column
                    
        elif next_move == 1:
            if move_column < m - 1 and relocation[move_line, move_column + 1] == " " and pher_food_map[move_line, move_column - 1] < pers_pher - 2:
                move_line = move_line
                move_column = move_column + 1
                   
        elif next_move == 2:
            if move_line < n - 1 and relocation[move_line + 1, move_column] == " " and pher_food_map[move_line, move_column - 1] < pers_pher - 2:
                move_line = move_line + 1
                move_column = move_column
                    
        elif next_move == 3:
            if move_column > 0 and relocation[move_line, move_column - 1] == " " and pher_food_map[move_line, move_column - 1] < pers_pher - 2:
                move_line = move_line
                move_column = move_column - 1
                                        
            #on déplace et modifie l'horloge de la fourmi
        clock1 = np.array(clock)
        clock = np.delete(clock1, m*move_line + move_column)
        clock = np.insert(clock, m*move_line + move_column, int(clock1[a, b] - 1))
        clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])  
        if a != move_line or b != move_column:
            clock = np.delete(clock, m*a + b)
            clock = np.insert(clock, m*a + b, 0)
            clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])     
                
            # on supprime l'indidu du monde
        relocation = np.delete(relocation, m*a + b) #on copie le monde sans l'individu
        relocation = np.insert(relocation, m*a + b, " ") #on remplace la place de l'individu par une case vide (avec un " ")
        relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                # on remplace la case vide par l'individu
        relocation = np.delete(relocation, m*(move_line) + move_column) #on copie le monde sans la case vide
        relocation = np.insert(relocation, m*(move_line) + move_column, "t") #on remplace la place de la case vide par l'individu (ant)
        relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
 
    return [relocation, pher_home_map, pher_food_map, clock]

def one_turn_eratic(map, map_pher_home, map_pher_food, clock_map):
    """Retourne les maps après un déplacement aléatoire de toute les fourmis"""
    one_map1 = map
    one_pher_home_map1 = map_pher_home
    one_pher_food_map1 = map_pher_food
    one_clock1 = clock_map
    one_map2 = map
    one_pher_home_map2 = map_pher_home
    one_pher_food_map2 = map_pher_food
    one_clock2 = clock_map
    one = [one_pher_home_map1, one_pher_food_map1, one_clock1, one_map1]
    for i in range(0, n):
        for j in range(0, m):
            if one_map1[i, j] != " ":
                a = one_eratic_move(one_map1, one_pher_home_map1, one_pher_food_map1, one_clock1, i, j)
                one_map2 = a[0]
                one_pher_home_map2 = a[1]
                one_pher_food_map2 = a[2]
                one_clock2 = a[3]
                one = a
                one_map1 = one_map2
                one_pher_home_map1 = one_pher_home_map2
                one_pher_food_map1 = one_pher_food_map2
                one_clock1 = one_clock2

    for i in range(0, n):
        for j in range(0, m):
            if one_map1[i, j] == "g":
                one_map1 = np.delete(one_map1, m*i + j)
                one_map1 = np.insert(one_map1, m*i + j, "f")
                one_map1 = np.array([[one_map1[m*k + l] for l in range(0, m)] for k in range(0, n)])
            elif one_map1[i, j] == "t":
                one_map1 = np.delete(one_map1, m*i + j)
                one_map1 = np.insert(one_map1, m*i + j, "s")
                one_map1 = np.array([[one_map1[m*k + l] for l in range(0, m)] for k in range(0, n)])
    return one



            
def simulation(map, map_pher_home, map_pher_food, food_map, clock_map, food_count, max_turn):
    """Effectue la simulation complète"""
    sim_map = map
    sim_pher_home = map_pher_home
    sim_pher_food = map_pher_food
    sim_food = food_map
    sim_clock = clock_map
    sim_count = food_count
    sim = [sim_map, sim_pher_home, sim_pher_food, sim_food, sim_clock, sim_count]
    for t in range(0, max_turn):
        a = one_turn(sim_map, sim_pher_home, sim_pher_food, sim_food, sim_clock, sim_count)
        sim_map = a[0]
        sim_pher_home = a[1]
        sim_pher_food = a[2]
        sim_food = a[3]
        sim_clock = a[4]
        sim_count = a[5]
        for i in range(0, n):
            for j in range(0, m):
                if sim_pher_home[i, j] > 0 :
                    sim_pher_home[i, j] = sim_pher_home[i, j] - 1
                if int(sim_pher_food[i, j]) > 0 :
                    sim_pher_food[i, j] = sim_pher_food[i, j] - 1
        print("tour :")
        print(t)
        print("sim_map :")
        print(sim_map)
        print("sim_pher_home :")
        print(sim_pher_home)
        print("sim_pher_food :")
        print(sim_pher_food)
        print("sim_food :")
        print(sim_food)
        print("sim_clock :")
        print(sim_clock)
        print("sim_count :")
        print(sim_count)
        if t % exploration_rate == 0:
            b = one_turn_eratic(sim_map, sim_pher_home, sim_pher_food, sim_clock)
            sim_map = b[0]
            sim_pher_home = b[1]
            sim_pher_food = b[2]
            sim_clock = b[3]
        sim = [sim_map, sim_pher_home, sim_pher_food, sim_food, sim_clock, sim_count]
    return sim

print(simulation(ants(map1, nb_ants), map_pher_home1, map_pher_food1, food_map1, clock_map(ants(map1, nb_ants)), food_count, max_turn))

        
        



[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'p' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'p' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' 'p' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' 'p' ' ' ' ' 'p' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'p' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' 'h' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' 'h' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' '

In [13]:
#Ces fonctions ne sont pas utilisées


# def one_turn(map):
    """Effectue un tour de la simulation complète"""
    move_map = map
    move_line = - 1
    move_column = - 1
    for i in range(0, n):
        for j in range(0, m):
             if move_map[i, j] == "f" : #la case sélectionnée est une fourmi
                next_move = random.randint(0, 3) #on décide le prochain mouvement au hasard
                if next_move == 0:
                    if i > 0 and move_map[i - 1, j] == " ": #la prochaine case existe et est vide
                        move_map = spatial_relocation(move_map, i, j, i - 1, j) #déplacement vers le haut
                        move_line = i - 1
                        move_column = j
                if next_move == 1:
                    if j < m - 1 and move_map[i, j + 1] == " ":
                        move_map = spatial_relocation(move_map, i, j, i, j + 1) #déplacement vers la droite
                        move_line = i
                        move_column = j + 1
                if next_move == 2:
                    if i < n - 1 and move_map[i + 1, j] == " ":
                        move_map = spatial_relocation(move_map, i, j, i + 1, j) #déplacement vers le bas
                        move_line = i + 1
                        move_column = j
                if next_move == 3:
                    if j > 0 and move_map[i, j - 1] == " ":
                        move_map = spatial_relocation(move_map, i, j, i, j - 1) #déplacement vers la gauche
                        move_line = i
                        move_column = j - 1
                if no_food(move_map, move_line, move_column) == False:
                    eratic_move = 0
                    next_move = 0
                    if move_map[move_line, move_column] == "f" :
                        move_map = np.delete(move_map, m*move_line + move_column) #on copie le monde sans l'individu
                        move_map = np.insert(move_map, m*move_line + move_column, "s") #on remplace la fourmi par une fourmi ayant de la nourriture (avec un "s")
                        move_map = np.array([[move_map[m*k + l] for l in range(0, m)] for k in range(0, n) ]) #on remet le monde sous forme de tableau
                        move_map = take_food(move_map, move_line, move_column)

                        # mouvement eratique périodique
                        if eratic_move > 2:
                            next_move = random.randint(0, 3) #on décide le procahin mouvement au hasard
                            if next_move == 0:
                                if line_found > 0 and move_map[line_found - 1, column_found] == " ": #la prochaine case existe et est vide
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found - 1, column_found) #déplacement vers le haut
                                    line_found = line_found - 1
                            if next_move == 1:
                                if column_found < m - 1 and move_map[line_found, column_found + 1] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found, column_found + 1) #déplacement vers la droite
                                    column_found = column_found + 1
                            if next_move == 2:
                                if line_found < n - 1 and move_map[line_found + 1, column_found] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found + 1, column_found) #déplacement vers le bas
                                    line_found = line_found + 1
                            if next_move == 3:
                                if column_found > 0 and move_map[line_found, column_found - 1] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found, column_found - 1) #déplacement vers la gauche
                                    column_found = column_found - 1
                            print(move_map)
                            eratic_move = 0
                    move_map = np.delete(move_map, m*line_found + column_found) #on copie le monde sans l'individu
                    move_map = np.insert(move_map, m*line_found + column_found, "f") #on remplace la fourmi par une fourmi sans nourriture (avec un "f")
                    move_map = np.array([[move_map[m*k + l] for l in range(0, m)] for k in range(0, n) ])
    return move_map

        print(move_map)
        print(food)
    return move_map
                    
def ants_food_quest(map):
    
    """Effectue le déplacement aléatoire de toutes les fourmis jusqu'à ce qu'il y en ait une qui trouve de la nourriture"""
    move_map = map
    move_line = - 1
    move_column = - 1
    food = False
    while food == False:
        for i in range(0, n):
            for j in range(0, m):
                if move_map[i, j] == "f" : #la case sélectionnée est une fourmi
                    next_move = random.randint(0, 3) #on décide le prochain mouvement au hasard
                    if next_move == 0:
                        if i > 0 and move_map[i - 1, j] == " ": #la prochaine case existe et est vide
                            move_map = spatial_relocation(move_map, i, j, i - 1, j) #déplacement vers le haut
                            move_line = i - 1
                            move_column = j
                    if next_move == 1:
                        if j < m - 1 and move_map[i, j + 1] == " ":
                            move_map = spatial_relocation(move_map, i, j, i, j + 1) #déplacement vers la droite
                            move_line = i
                            move_column = j + 1
                    if next_move == 2:
                        if i < n - 1 and move_map[i + 1, j] == " ":
                            move_map = spatial_relocation(move_map, i, j, i + 1, j) #déplacement vers le bas
                            move_line = i + 1
                            move_column = j
                    if next_move == 3:
                        if j > 0 and move_map[i, j - 1] == " ":
                            move_map = spatial_relocation(move_map, i, j, i, j - 1) #déplacement vers la gauche
                            move_line = i
                            move_column = j - 1
                    if no_food(move_map, move_line, move_column) == False:
                        food = True
        print(move_map)
        print(food)
    return move_map

#print(one_food_quest(ants(generate_simple_map(n, m), 1), 3, 3))
#print(ants_food_quest(ants(generate_simple_map(m, n), 10)))

def return_home (map):
    """Si une fourmi a trouvé de la nourriture, effectue le déplacement de celle-ci vers la fourmilière"""
    move_map = map
    line_found = - 1 #ligne de la fourmi découvreuse de nourriture
    column_found = - 1 #colonne de la fourmi découvreuse de nourriture
    eratic_move = 0
    next_move = 0
    for i in range(0, n):
        for j in range(0, m):
            if move_map[i, j] == "f" :
                if no_food(move_map, i, j) == False:
                    line_found = i
                    column_found = j
                    move_map = np.delete(move_map, m*line_found + column_found) #on copie le monde sans l'individu
                    move_map = np.insert(move_map, m*line_found + column_found, "s") #on remplace la fourmi par une fourmi ayant de la nourriture (avec un "s")
                    move_map = np.array([[move_map[m*k + l] for l in range(0, m)] for k in range(0, n) ]) #on remet le monde sous forme de tableau
                    move_map = take_food(move_map, line_found, column_found)
                    while no_home(move_map, line_found, column_found):
                        # mouvement diagonal vers la fourmilière
                        if line_found >= n/2 and column_found >= m/2 and move_map[line_found - 1, column_found - 1] == " ": #la fourmi est dans le quart bas droit de la map
                            move_map = spatial_relocation(move_map, line_found, column_found, line_found - 1, column_found - 1)
                            line_found = line_found - 1
                            column_found = column_found - 1
                        if line_found >= n/2 and column_found <= m/2 and move_map[line_found - 1, column_found + 1] == " ": #la fourmi est dans le quart bas gauche de la map
                            move_map = spatial_relocation(move_map, line_found, column_found, line_found - 1, column_found + 1)
                            line_found = line_found - 1
                            column_found = column_found + 1  
                        if line_found <= n/2 and column_found <= m/2 and move_map[line_found + 1, column_found + 1] == " ": #la fourmi est dans le quart haut gauche de la map
                            move_map = spatial_relocation(move_map, line_found, column_found, line_found + 1, column_found + 1)
                            line_found = line_found + 1
                            column_found = column_found + 1   
                        if line_found <= n/2 and column_found >= m/2 and move_map[line_found + 1, column_found - 1] == " ": #la fourmi est dans le quart haut droit de la map
                            move_map = spatial_relocation(move_map, line_found, column_found, line_found + 1, column_found - 1)
                            line_found = line_found + 1
                            column_found = column_found - 1   
                        eratic_move = eratic_move + 1
                        print(move_map)
                    
                        # mouvement eratique périodique
                        if eratic_move > 2:
                            next_move = random.randint(0, 3) #on décide le procahin mouvement au hasard
                            if next_move == 0:
                                if line_found > 0 and move_map[line_found - 1, column_found] == " ": #la prochaine case existe et est vide
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found - 1, column_found) #déplacement vers le haut
                                    line_found = line_found - 1
                            if next_move == 1:
                                if column_found < m - 1 and move_map[line_found, column_found + 1] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found, column_found + 1) #déplacement vers la droite
                                    column_found = column_found + 1
                            if next_move == 2:
                                if line_found < n - 1 and move_map[line_found + 1, column_found] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found + 1, column_found) #déplacement vers le bas
                                    line_found = line_found + 1
                            if next_move == 3:
                                if column_found > 0 and move_map[line_found, column_found - 1] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found, column_found - 1) #déplacement vers la gauche
                                    column_found = column_found - 1
                            print(move_map)
                            eratic_move = 0
                    print(1)
                    move_map = np.delete(move_map, m*line_found + column_found) #on copie le monde sans l'individu
                    move_map = np.insert(move_map, m*line_found + column_found, "f") #on remplace la fourmi par une fourmi sans nourriture (avec un "f")
                    move_map = np.array([[move_map[m*k + l] for l in range(0, m)] for k in range(0, n) ])
    return move_map

#print(return_home(ants_food_quest(ants(map, 10))))
def one_food_quest (map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Si la case [a, b] est une fourmi, renvoie la matrice après son déplacement aléatoire sans retourner sur ses pas
    sinon renvoie la matrice initiale"""
    compt = 20
    move_map = map
    move_line = a #ligne de la fourmi
    move_column = b #colonne de la fourmi
    previous_location = - 10 #on retient la position précédente pour ne pas y revenir
    if move_map[a, b] == "f" : #la case sélectionnée est une fourmi
        while no_food(move_map, move_line, move_column) and compt > 0: #il n'y a pas de nourriture à coté de la fourmi            
            next_move = random.randint(0, 3) #on décide le procahin mouvement au hasard
            if next_move == 0 and previous_location != 2:
                if move_line > 0 and move_map[move_line - 1, move_column] == " ": #la prochaine case existe et est vide
                    move_map = spatial_relocation(move_map, move_line, move_column, move_line - 1, move_column) #déplacement vers le haut
                    previous_location = next_move
                    move_line = move_line - 1
            if next_move == 1 and previous_location != 3:
                if move_column < m - 1 and move_map[move_line, move_column + 1] == " ":
                    move_map = spatial_relocation(move_map, move_line, move_column, move_line, move_column + 1) #déplacement vers la droite
                    previous_location = next_move                  
                    move_column = move_column + 1
            if next_move == 2 and previous_location != 0:
                if move_line < n - 1 and move_map[move_line + 1, move_column] == " ":
                    move_map = spatial_relocation(move_map, move_line, move_column, move_line + 1, move_column) #déplacement vers le bas
                    previous_location = next_move
                    move_line = move_line + 1
            if next_move == 3 and previous_location != 1:
                if move_column > 0 and move_map[move_line, move_column - 1] == " ":
                    move_map = spatial_relocation(move_map, move_line, move_column, move_line, move_column - 1) #déplacement vers la gauche
                    previous_location = next_move
                    move_column = move_column - 1
            print(no_food(move_map, move_line, move_column), move_line, move_column)
            compt = compt - 1
            print(move_map)
    return move_map


IndentationError: unexpected indent (<ipython-input-13-3a745dd561c4>, line 5)

In [7]:
a = 6
b = 8
c = 10
list1 = [b, c, a]
sorted(list1)[1]

8

In [4]:


def create_food(map) :
    """Créer un point de nourriture sur le monde map"""
    global val_food
    food_line = random.randint(0, 2*n//5 - 1) # on définit ainsi la périphérie comme étant les cases du bord de la matrice avec un largeur de n/5
    food_column = random.randint(0, 2*m//5 - 1)
    if food_line > n//5 - 1: # on est sur la périphérie du bas
        food_line = n - 2 - n//5 + food_line
    if food_column > m//5 - 1: # on est sur la périphérie de droite
        food_column = m - 2 - m//5 + food_column
    where = random.sample([food_line, food_column], 1) # on définit une périphérie où se mettre (horizontale ou verticale)
    if where == food_line: # on est sur une périphérie en haut ou en bas
        food_column = random.randint(0, m - 1) # on définit une case au hasard sur cette périphérie
    else : #on est sur une périphérie latérale
        food_line = random.randint(0, n - 1) # on définit une case au hasard sur cette périphérie
    # on place dans la matrice de point de nourriture de quantité aléatoire food_quantity et de position périphérique aléatoire (food_line, food_column)
    food_quantity = random.randint(1, max_food_quantity) # on attribue à ce point de nourriture une certaine quantité de nourriture
    print(food_quantity)
    map = np.delete(map, m*food_line + food_column)
    map = np.insert(map, m*food_line + food_column, "n")
    map = np.array([[map[m*j + i] for i in range(0, m)] for j in range(0, n) ])
    val_food = np.delete(val_food, m*food_line + food_column)
    val_food = np.insert(val_food, m*food_line + food_column, food_quantity)
    val_food = np.array([[val_food[m*j + i] for i in range(0, m)] for j in range(0, n) ])
    print(val_food)
    return map

print(create_food(map))



7


IndexError: index 162 is out of bounds for axis 0 with size 1

In [19]:
list1 = (45, 78, 9, 90)
print(sorted(list1))

[9, 45, 78, 90]
